Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [28]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
ESC_KEY = 27

In [29]:
def get_video_data(filename):
    video = cv2.VideoCapture(filename)
    fps = round(video.get(cv2.CAP_PROP_FPS))
    framerate = round(1000 / fps) # 1000 ms / FPS
    resolution = (round(video.get(cv2.CAP_PROP_FRAME_WIDTH)),round(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video.release()
    print(framerate)
    print(fps)
    print(resolution)
    video.release()
    return framerate, fps, resolution

In [30]:
FRAMERATE, FPS, RESOLUTION = get_video_data(VFILENAME)
WIDTH, HEIGHT = RESOLUTION

33
30
(722, 406)


Load video frames from file

In [31]:
def get_frames(filename):
    video = cv2.VideoCapture(filename) # Open Video
    while video.isOpened():
        retrieved, frame = video.read() # Read frames
        if retrieved:
            yield frame 
        else:
            break
    video.release() # Video is closed, release resources
    yield None

Creating a particle cloud

In [32]:
NUM_PARTICLES = 5000
VEL_RANGE = 0.5
# particles = [particle][(x,y)][velocityX][velocityY]
def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES,4) # 5000 Rows, 4 Columns, Number between 0 and 1
    particles = particles * np.array((WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE)) # Range of X, Y, and velocities for every particle
    particles[:,2:4] -= VEL_RANGE/2.0 # Center Velocity at 0
    print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [33]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [34]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = min(WIDTH - 1, particles[i,0]) # Prevents x value from being greater than WIDTH
        particles[i,1] = min(HEIGHT -1, particles[i,1]) # Prevents y value from being greater than HEIGHT 
    return particles

Measure each particle's quality

In [35]:
TARGET_COLOR = np.array((156,74,38)) # BGR Color Value
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_color = frame[y,x,:] # y,x, BGR
        errors[i] = np.sum((TARGET_COLOR - pixel_color) ** 2) # Difference in color values
    return errors

Assign weights to the particles based on their quality of match

In [36]:
def compute_weights(errors):
    weights = np.max(errors) - errors # Invert errors (Highest error = Lowest Weight)
    # Set edge particles weight to 0
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) | 
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT - 1)
    ] = 0.0
    return weights

Resample particles according to their weights

In [37]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights) # Normalize
    
    # New particles based on weights (High Weight = High Chance)
    index_numbers = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p=probabilities)
    particles = particles[index_numbers,:] # Replace particles with chosen indices
    
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x),int(y))

Fuzz the particles

In [38]:
def apply_noise(particles):
    return particles

Display the video frames

In [39]:
def display(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    cv2.imshow('frame',frame)
    if cv2.waitKey(30) == ESC_KEY: # Pause Video
        if cv2.waitKey(0) == ESC_KEY: # Exit Video
            return True
            
    return False

Main routine

In [40]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 3.96243350e+02  2.90366883e+02  5.13816880e-02  2.24415915e-02]
 [ 3.05878765e+02  2.62233010e+02 -3.12063944e-02  1.95886500e-01]
 [ 6.95764513e+02  1.55677257e+02  1.45862519e-01  1.44474599e-02]
 [ 4.10128173e+02  3.75792235e+02 -2.14481971e-01 -2.06435350e-01]
 [ 1.45976830e+01  3.38043657e+02  1.39078375e-01  1.85006074e-01]
 [ 7.06562443e+02  3.24458377e+02 -1.92603189e-02  1.40264588e-01]
 [ 8.53941355e+01  2.59807935e+02 -1.78323356e-01  2.22334459e-01]
 [ 3.76774488e+02  1.68352748e+02 -1.17722194e-01  1.37116845e-01]
 [ 3.29340540e+02  2.30784183e+02 -2.40605100e-01  5.88177485e-02]
 [ 4.41933112e+02  2.50475203e+02  2.21874039e-01  9.09101496e-02]
 [ 2.59564704e+02  1.77434973e+02  9.88155980e-02 -2.19887264e-01]
 [ 4.81405569e+02  2.72278975e+02 -1.44808719e-01 -1.85536851e-01]
 [ 2.27739269e+02  1.47666573e+02  3.50983852e-02 -3.06992433e-02]
 [ 7.13605911e+02  4.14301932e+01 -1.45561622e-01 -1.69345241e-01]
 [ 4.71544211e+02  1.02836391e+02 -1.68446136e-02 -1.27787204e

C:\Users\Michael\AppData\Local\Temp\ipykernel_32648\463983529.py:2: RuntimeWarning: invalid value encountered in divide
  probabilities = weights / np.sum(weights) # Normalize


ValueError: probabilities contain NaN

In [61]:
fourcc = cv2.VideoWriter_fourcc('M','P','4','V') # MP4V Format Output
video_out = cv2.VideoWriter("walking-output.mp4",fourcc,FPS,RESOLUTION) # (filename,fourcc,frames per sec, resolution)

def save_video(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    video_out.write(frame)
            
    return False

In [62]:
# Save Video to Disk

particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: 
        break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    save_video(frame, particles, location)
    
cv2.destroyAllWindows()
video_out.release()


[[ 2.66603066e+02  8.57983573e+01 -1.15476157e-02 -2.08882822e-01]
 [ 1.71590062e+02  3.39000967e+01 -6.11240201e-02 -1.34422830e-01]
 [ 6.49246985e+02  7.00101504e+01  4.30303163e-02 -2.35340199e-01]
 [ 1.59680343e+02  3.08734577e+02 -1.20935525e-01 -1.80093242e-01]
 [ 9.06419609e+01  1.91234088e+02 -5.21805162e-02  7.66441127e-02]
 [ 4.04460185e+02  3.17367098e+01 -2.36418272e-01  2.99999244e-03]
 [ 6.54100625e+02  1.22942816e+02  5.32802877e-02  1.85965287e-01]
 [ 6.79615671e+02  1.91832310e+01 -1.40021216e-01 -1.66106231e-01]
 [ 7.19240866e+02  2.67135095e+02 -9.41397579e-02 -1.20006111e-02]
 [ 2.40833451e+02  3.65229143e+01  1.32753254e-01 -1.22292276e-01]
 [ 3.51775794e+02  2.42106347e+02  1.51771657e-01  8.37211702e-02]
 [ 3.69970317e+02  5.83611996e+01 -5.04871552e-02 -1.90857125e-01]
 [ 7.85009894e+01  2.11790643e+02  1.21228745e-02  1.46274026e-02]
 [ 3.42818191e+02  6.25784959e+01  1.13429573e-01  1.14525328e-01]
 [ 3.74393915e+02  1.32625690e+02  3.74000496e-02 -6.09580182e